In [1]:
!wget -O Iris.csv https://raw.githubusercontent.com/selva86/datasets/master/Iris.csv


--2024-11-01 00:30:53--  https://raw.githubusercontent.com/selva86/datasets/master/Iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5107 (5.0K) [text/plain]
Saving to: ‘Iris.csv’

Iris.csv            100%[===================>]   4.99K  --.-KB/s    in 0s      

2024-11-01 00:30:53 (49.3 MB/s) - ‘Iris.csv’ saved [5107/5107]



In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!mv Iris.csv /content/drive/MyDrive/


In [5]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler


In [6]:
spark = SparkSession.builder.appName("IrisDecisionTree").getOrCreate()


In [8]:
data = spark.read.csv("/content/drive/MyDrive/Iris.csv", header=True, inferSchema=True)



In [10]:
assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(data).select("Species", "features")


train, test = data_with_features.randomSplit([0.8, 0.2], seed=42)


In [14]:
from pyspark.ml.feature import StringIndexer


indexer = StringIndexer(inputCol="Species", outputCol="label")
data_with_labels = indexer.fit(data_with_features).transform(data_with_features)



In [15]:
train, test = data_with_labels.randomSplit([0.8, 0.2], seed=42)
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model = dt.fit(train)

In [16]:
predictions = model.transform(test)

In [17]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")


accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")


precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")


recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")


f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")

spark.stop()

Accuracy: 0.9166666666666666
Precision: 0.9166666666666666
Recall: 0.9166666666666666
F1 Score: 0.9166666666666667
